<h1>AgigrateOne Shipping Intelligence</h1>

<h2>Connect to PosgreSQL database</h2>

In [51]:
import psycopg2

# Connection parameters
dbname = "postgres"
user = "postgres"
password = "postgres"
host = "localhost"
port = "5432"

# Establish a connection to the database
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
        )
    print("Connected to the database!")

    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error connecting to the database:", e)

Connected to the database!


<h2>2.2.1 How many unique ships do we have data on?</h2>
<h4>https://sinay.ai/en/what-is-the-difference-between-imo-and-mmsi/</h4>

In [53]:
cur.execute("SELECT COUNT(DISTINCT v.imo) FROM data_science.vessel_info AS v")
result = cur.fetchall()
print(f"There are {result} ships on data.")

There are [(2276,)] ships on data.


<h2>2.2.2 At present, we have data on numerous container ships. <br> 
    How many of these ships would be classified as ULCV?</h2>
<h4>Ultra Large Container Vessel (ULCV)</h4>
<h5>With a length of 366 m, a beam of 49 m, draft of 15 m, and a capacity of 18,270 TEU</h5>

In [54]:
cur.execute("SELECT DISTINCT v.imo,  v.vessel_name, v.length, v.width FROM data_science.vessel_info AS v WHERE v.length >= 366 AND v.width >= 49")
rows = cur.fetchall()
for row in rows:
    print(row)  # Output each row as a tuple

('9321483', 'EMMA MAERSK', 397.71, 56.55)
('9321495', 'ESTELLE MAERSK', 397.71, 56.55)
('9321500', 'ELEONORA MAERSK', 398.9, 56.55)
('9321512', 'EVELYN MAERSK', 398.9, 56.4)
('9321524', 'EBBA MAERSK', 397.71, 56.55)
('9321536', 'ELLY MAERSK', 398.9, 56.55)
('9321548', 'EDITH MAERSK', 397.71, 56.55)
('9321550', 'EUGEN MAERSK', 398.9, 56.55)
('9399002', 'MSC DANIELA', 366.07, 51.24)
('9399014', 'MSC BEATRICE', 366.07, 51.24)
('9399026', 'MSC KALINA', 366.07, 51.24)
('9399038', 'MSC BETTINA', 366.07, 51.24)
('9399040', 'MSC IRENE', 366.07, 51.24)
('9399052', 'MSC EMANUELA', 366.07, 51.24)
('9401130', 'MSC EVA', 366.06, 51.24)
('9401142', 'MSC GAIA', 366.03, 51.24)
('9454436', 'CMA CGM MARCO POLO', 396.0, 53.6)
('9466867', 'CSCL STAR', 366.07, 51.2)
('9467275', 'CSCL MERCURY', 366.06, 51.23)
('9467287', 'CSCL MARS', 366.07, 51.23)
('9467299', 'CSCL SATURN', 366.07, 51.23)
('9467316', 'CSCL NEPTUNE', 366.04, 51.23)
('9516428', 'COSCO ENGLAND', 366.0, 51.2)
('9525912', 'AL RIFFA', 366.04, 51

In [55]:
cur.execute("SELECT COUNT(DISTINCT v.imo) FROM data_science.vessel_info AS v WHERE v.length >= 366 AND v.width >= 49")
result = cur.fetchall()
print(f"There are {result} ULCV vessels on data")

There are [(185,)] ULCV vessels on data


<h2>2.2.3 After analyzing the 'ports' table, you noticed that erroneous information has been entered into the 'code' column. <br>
    Utilizing an alternative column in the table, retrieve a list of rows where most 'code' values are incorrect.</h2>
<h4>UN/LOCODEs, are unique five-character codes assigned by the United Nations Economic Commission for Europe (UNECE)<br>
These codes consist of two letters representing the country and three letters representing the port city or location.</h4>

In [56]:
# Function to format a column value with padding
def format_column_value(value, width):
    if len(value) > width:
        return value[:width]  # Truncate the string if it exceeds the width
    else:
        return value.ljust(width)  # Pad the string with spaces if it's shorter

# Run SQL statement
cur.execute("SELECT p.port, p.country, p.country_code, p.code FROM data_science.vessel_ports AS p WHERE SUBSTRING(p.code FROM 1 FOR 2) != p.country_code")

# Get column names (headings)
col_names = [desc[0] for desc in cur.description]

# Define maximum width for each column
max_width = 20  # Adjust the maximum width as needed

# Display the headings with padding
formatted_col_names = [format_column_value(name, max_width) for name in col_names]
print("\t".join(formatted_col_names))  # Assuming tab-separated output

# Fetch all rows from the result set
rows = cur.fetchall()

# Display the output as a table with padding for each column value
for row in rows:
    formatted_row = [format_column_value(str(value), max_width) for value in row]
    print("\t".join(formatted_row))  # Assuming tab-separated output

port                	country             	country_code        	code                
PIRAEUS ANCH (Anchor	Greece (GR)         	GR                  	16                  
HUNTERS QUAY        	United Kingdom (GB) 	GB                  	4                   
MCINROY             	United Kingdom (GB) 	GB                  	0                   
FLISVOS (Marina/Loca	Greece (GR)         	GR                  	85                  
SNAPTUN             	Denmark (DK)        	DK                  	5                   
ILE DE BATZ         	France (FR)         	FR                  	3                   
ORIENT POINT        	USA (US)            	US                  	2                   
PLUM ISLAND         	USA (US)            	US                  	0                   
JUDITH POINT        	USA (US)            	US                  	42                  
BJORLANDA KILE (Mari	Sweden (SE)         	SE                  	8                   
MORDOGAN            	Turkey (TR)         	TR                  	0            

<h2>Close database connection</h2>

In [57]:
cur.close()
conn.close()